In [ ]:
# Set openai key
# Please gen and add `OPENAI_API_KEY` in the `.env` file in the current directory
# https://platform.openai.com/account/api-keys
import os

from dotenv import load_dotenv

load_dotenv()

### Models

In [ ]:
LLM_MODEL = "gpt-3.5-turbo"

In [ ]:
# Creating OpenAI chat
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model=LLM_MODEL)

In [ ]:
# Simple messages
from langchain.schema import HumanMessage

question = "When did Elon Musk buy twitter?"

messages = [HumanMessage(content=question)]
chat(messages)

### Prompts

In [ ]:
# Creating prompt template
from langchain.prompts import ChatPromptTemplate

question_template = """
I was hoping you could provide me a detailed recipe dish that I can make from only the following ingredients:
{ingredients}.
Use all listed ingredients.
"""

prompt_template = ChatPromptTemplate.from_template(question_template)

In [ ]:
# Generate questions to OpenAI from template

ingredients = [
    "1 egg",
    "1 bottle of coke",
    "chiken",
    "salt",
    "basil",
    "coconut milk 200 ml",
]

messages = prompt_template.format_messages(ingredients=", ".join(ingredients))

print(messages[0].content)

In [ ]:
# Ask OpenAI
response = chat(messages)
print(response.content)

In [ ]:
# A more complicated prompt template

raw_template = """
Translate this text that is delimited by triple backticks from {original_language} to {destination_language}.
After translation make a short description in the {style_type} style.
```{text}```
"""

original_text = """
メロスは怒った。悪い王様を許せないと思いました。メロスは難しいことがわかりません。メロスは村で楽しく生活をしていました。しかし、メロスは悪いことを許せません。メロスは村からシクラスという街に来ました。メロスにはお父さんもお母さんもいません。奥さんもいません。１６歳の妹と二人で生活をしています。妹はもうすぐ結婚します。メロスは結婚式の用意のために買い物をしに街に来たのです。街を歩いていると、メロスは街がおかしいと思いました。前に来た時より、街がとても静かなのです。そこで、メロスは街の人に聞きました。「なぜ街が静かなのですか？」
"""

prompt_template = ChatPromptTemplate.from_template(raw_template)

messages = prompt_template.format_messages(
    original_language="Japanese",
    destination_language="English",
    style_type="Pirate",
    text=original_text,
)

In [ ]:
messages[0].content

In [ ]:
# Ask OpenAI
response = chat(messages)
print(response.content)

### Output parsers

In [ ]:
# Template includes output format
cv_template = """
Parse CV and extract the following information from it.
Format the output as JSON with the following keys:
title - candidate's title (string)
experience - candidate's experience in years (number)
projects - list of projects the candidate worked on (list of objects with keys: project_name and company)
skills - list of candidate's skills (list of strings)
short_summary - what tasks the candidate worked on the most (string)
```{cv}```
"""

cv = """
Junior Python Developer
Tallahassee, Florida • +1-234-456-789 • professionalemail@resumeworded.com • linkedin.com/in/username

Resume Worded, New York, NY 09/2015 – Present
Augmented reality startup with 50+ employees and $100m+ annual revenue

Junior Python Developer
1. Collaborated with a 10-member team to design a content aggregator and expense tracker that provided
solutions with 71% code churn.
2. Developed the API, backend, and frontend for a single-page application within the first 30+ days of joining
the team.
3. Created views and templates using Python and Django's view controller and templating language to develop
10+ user-friendly interfaces.
4. Modified functions, cursors, queries, triggers, and stored procedures for MySQL database while processing
data; improved system performance by 84%.
Polyhire, London, United Kingdom 10/2012 – 08/2015
Provides quality assurance and control testing for global markets

Software Tester
1. Created a testing framework that boosted efficiency in detecting errors before release, improving the
company's software production schedule by 58%.
2. Uncovered security vulnerabilities in 10+ applications via manual review and automated tools; escalated
findings to RW's software security unit.
3. Detected a critical defect in some production codes that saved the company from a $200K flawed
investment.
Growthsi, London, United Kingdom 07/2010 – 09/2012
Career training and membership SaaS with 150,000 paying users
SQL DBA
4. Consolidated 30+ SQL Server databases into 20 and merged all company hardware, reducing 33% of the
original hardware acquisition cost.
5. Explained backup policies of data to senior management during a 15-day seminar that non-technical
attendees attended.
6. Controlled access to applications and data by 2100+ users, saving $30K in quarterly licensing fees in the
first year of employment.
Disaster Recovery Specialist, ABC Company, London, United Kingdom 11/2008 – 12/2009
Coder, XYZ Company, London, United Kingdom 07/2005 – 10/2008
EDUCATION
_______________________________________________________________________________________________________________
Resume Worded University, New York, NY 06/2005
Associate of Applied Science — Computer Programming
SKILLS
_______________________________________________________________________________________________________________
Web Technologies: HTML (Advanced), CSS (Experienced), Angular JS, Jira, Django 3.0, GIT Hub, MySQL, Javascript
Techniques: Software Development, Amazon Web Services (AWS), ORM Libraries, Agile frameworks, OOPS
"""

prompt_template = ChatPromptTemplate.from_template(cv_template)

messages = prompt_template.format_messages(cv=cv)

In [ ]:
# Ask OpenAI
response = chat(messages)
parsed_cv = response.content
print(parsed_cv)

In [ ]:
# parsed_cv is a string, not a python dictionary
type(parsed_cv)

#### Structured output parser​

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser


title = ResponseSchema(
    name="title",
    description="candidate's title",
    type="string",
)
experience = ResponseSchema(
    name="experience",
    description="candidate's experience in years",
    type="integer",
)
projects = ResponseSchema(
    name="projects",
    description="list of projects the candidate worked on",
    type="list of objects with keys: project_name and company",
)
skills = ResponseSchema(
    name="skills",
    description="list of candidate's skills",
    type="list of strings",
)
short_summary = ResponseSchema(
    name="short_summary",
    description="what tasks the candidate worked on the most",
    type="string",
)

response_schemas = [
    title,
    experience,
    projects,
    skills,
    short_summary,
]

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
print(output_parser)

In [ ]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

In [ ]:
cv_template_2 = """
Parse CV and extract the following information from it.
{format_instructions}
```{cv}```
"""

prompt_template_2 = ChatPromptTemplate.from_template(template=cv_template_2)

messages_2 = prompt_template_2.format_messages(
    cv=cv,
    format_instructions=format_instructions,  # format instruction created above
)

In [ ]:
print(messages_2[0].content)

In [ ]:
# Ask OpenAI
response = chat(messages)
parsed_cv_2 = response.content
print(parsed_cv_2)

In [ ]:
type(parsed_cv_2)

In [ ]:
output_dict = output_parser.parse(parsed_cv_2)
print(output_dict)

#### Datetime parser

In [ ]:
from langchain.output_parsers import DatetimeOutputParser

datetim_output_parser = DatetimeOutputParser()
template = """
{question}

{format_instructions}
"""
prompt = ChatPromptTemplate.from_template(template)
messages = prompt.format_messages(
    question="When was openAI launched?",
    format_instructions=datetim_output_parser.get_format_instructions(),
)

In [ ]:
print(messages[0].content)

In [ ]:
# Ask OpenAI
response = chat(messages)
response.content

In [ ]:
# Answer has wrong format
datetim_output_parser.parse(response.content)

#### Fixing parser

In [ ]:
from langchain.output_parsers import OutputFixingParser

fix_parser = OutputFixingParser.from_llm(parser=datetim_output_parser, llm=ChatOpenAI())
fix_parser.parse(response.content)